<a href="https://colab.research.google.com/github/ShyamSanthosh04/Liver-Cancer-Tumor-Detection/blob/main/Clean_Tumor_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


from google.colab import files
files.upload()

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d falahgatea/liver-cancer-dataset

In [ ]:
import zipfile
import os

zip_path = '/content/liver-cancer-dataset.zip'
output_path = '/content/output'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(output_path)

In [ ]:

import os
import nibabel as nib
import numpy as np
from sklearn.model_selection import train_test_split
from PIL import Image
import cv2

volume_dirs = ['/content/output/volume_pt1', '/content/output/volume_pt2',
               '/content/output/volume_pt3', '/content/output/volume_pt4',
               '/content/output/volume_pt5']
segmentation_dir = '/content/output/segmentations'

output_dir = '/content/dataset/processed'
tumor_dir = os.path.join(output_dir, 'Tumor')
no_tumor_dir = os.path.join(output_dir, 'NoTumor')

os.makedirs(tumor_dir, exist_ok=True)
os.makedirs(no_tumor_dir, exist_ok=True)

image_size = (224, 224)
min_tumor_area = 300

def load_nifti(file_path):
    img = nib.load(file_path)
    return img.get_fdata()

def process_data():
    for vol_dir in volume_dirs:
        for vol_file in os.listdir(vol_dir):
            if vol_file.endswith('.nii'):
                volume_path = os.path.join(vol_dir, vol_file)
                volume_data = load_nifti(volume_path)

                seg_file = vol_file.replace('volume', 'segmentation')
                seg_path = os.path.join(segmentation_dir, seg_file)
                seg_data = load_nifti(seg_path)

                for i in range(volume_data.shape[2]):
                    volume_slice = volume_data[:, :, i]
                    seg_slice = seg_data[:, :, i]

                    tumor_area = np.sum(seg_slice > 0)
                    if tumor_area > min_tumor_area:
                        output_path = os.path.join(tumor_dir, f"{vol_file}_slice_{i}.png")
                    else:
                        output_path = os.path.join(no_tumor_dir, f"{vol_file}_slice_{i}.png")

                    resized_slice = cv2.resize(volume_slice, image_size)
                    resized_slice = np.interp(resized_slice, (resized_slice.min(), resized_slice.max()), (0, 255)).astype(np.uint8)

                    img = Image.fromarray(resized_slice)

                    img.save(output_path)

def split_data():
    images = []
    labels = []

    for label_dir, label in [(tumor_dir, 1), (no_tumor_dir, 0)]:
        for img_file in os.listdir(label_dir):
            if img_file.endswith('.png'):
                images.append(os.path.join(label_dir, img_file))
                labels.append(label)

    X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

    train_dir = os.path.join(output_dir, 'train')
    test_dir = os.path.join(output_dir, 'test')
    os.makedirs(train_dir, exist_ok=True)
    os.makedirs(test_dir, exist_ok=True)

    for img, label in zip(X_train, y_train):
        dest_dir = os.path.join(train_dir, 'Tumor' if label == 1 else 'NoTumor')
        os.makedirs(dest_dir, exist_ok=True)
        os.rename(img, os.path.join(dest_dir, os.path.basename(img)))

    for img, label in zip(X_test, y_test):
        dest_dir = os.path.join(test_dir, 'Tumor' if label == 1 else 'NoTumor')
        os.makedirs(dest_dir, exist_ok=True)
        os.rename(img, os.path.join(dest_dir, os.path.basename(img)))

process_data()
split_data()

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_dir = '/content/dataset/processed/train'
test_dir = '/content/dataset/processed/test'

train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Input

def build_custom_cnn():
    model = Sequential([
        Input(shape=(224, 224, 3)),
        Conv2D(32, (3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
cnn_model = build_custom_cnn()
cnn_history = cnn_model.fit(
    train_generator,
    epochs=25,
    validation_data=test_generator
)

cnn_loss, cnn_acc = cnn_model.evaluate(test_generator)
print(f"Custom CNN Test Accuracy: {cnn_acc * 100:.2f}%")

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import numpy as np

train_dir = '/content/dataset/processed/train'
test_dir = '/content/dataset/processed/test'

train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

img_size = (224, 224)
batch_size = 32

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False
)

def build_custom_cnn():
    model = Sequential([
        Input(shape=(224, 224, 3)),
        Conv2D(32, (3,3), activation='relu'),
        MaxPooling2D(),
        Conv2D(64, (3,3), activation='relu'),
        MaxPooling2D(),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

def build_alexnet():
    model = Sequential([
        Input(shape=(224,224,3)),
        Conv2D(96, (11,11), strides=4, activation='relu'),
        MaxPooling2D(pool_size=(3,3), strides=2),
        Conv2D(256, (5,5), padding="same", activation='relu'),
        MaxPooling2D(pool_size=(3,3), strides=2),
        Conv2D(384, (3,3), padding="same", activation='relu'),
        Conv2D(384, (3,3), padding="same", activation='relu'),
        Conv2D(256, (3,3), padding="same", activation='relu'),
        MaxPooling2D(pool_size=(3,3), strides=2),
        Flatten(),
        Dense(4096, activation='relu'),
        Dropout(0.5),
        Dense(4096, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

def build_vgg16():
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224,224,3))
    base_model.trainable = False
    model = Sequential([
        base_model,
        Flatten(),
        Dense(256, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

cnn_model = build_custom_cnn()
cnn_history = cnn_model.fit(
    train_generator,
    epochs=25,
    validation_data=test_generator
)

alexnet_model = build_alexnet()
alexnet_history = alexnet_model.fit(
    train_generator,
    epochs=25,
    validation_data=test_generator
)

vgg16_model = build_vgg16()
vgg16_history = vgg16_model.fit(
    train_generator,
    epochs=25,
    validation_data=test_generator
)

def plot_accuracy(history, model_name):
    plt.figure(figsize=(12,5))

    plt.subplot(1,2,1)
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Val Accuracy')
    plt.title(f'{model_name} - Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.subplot(1,2,2)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Val Loss')
    plt.title(f'{model_name} - Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    plt.tight_layout()
    plt.show()

plot_accuracy(cnn_history, "Custom CNN")
plot_accuracy(alexnet_history, "AlexNet")
plot_accuracy(vgg16_history, "VGG16")

cnn_eval = cnn_model.evaluate(test_generator, verbose=0)
alexnet_eval = alexnet_model.evaluate(test_generator, verbose=0)
vgg16_eval = vgg16_model.evaluate(test_generator, verbose=0)

print("\n📊 Final Test Accuracies:")
print(f"Custom CNN:  {cnn_eval[1]*100:.2f}%")
print(f"AlexNet:     {alexnet_eval[1]*100:.2f}%")
print(f"VGG16:       {vgg16_eval[1]*100:.2f}%")

In [ ]:
import os
import nibabel as nib
import numpy as np
import cv2
from tqdm import tqdm

volume_paths = [
    '/content/output/volume_pt1',
    '/content/output/volume_pt2',
    '/content/output/volume_pt3',
    '/content/output/volume_pt4',
    '/content/output/volume_pt5'
]
segmentation_path = '/content/output/segmentations'

output_images_folder = '/content/rcnn_dataset/images'
output_labels_folder = '/content/rcnn_dataset/labels'

os.makedirs(output_images_folder, exist_ok=True)
os.makedirs(output_labels_folder, exist_ok=True)

def extract_bounding_boxes(mask):
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    boxes = []
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        boxes.append([x, y, x+w, y+h])
    return boxes

image_counter = 0
for volume_dir in volume_paths:
    volume_files = os.listdir(volume_dir)

    for file_name in tqdm(volume_files):
        volume_file = os.path.join(volume_dir, file_name)
        segmentation_file = os.path.join(segmentation_path, file_name)

        if not os.path.exists(segmentation_file):
            continue

        volume = nib.load(volume_file).get_fdata()
        segmentation = nib.load(segmentation_file).get_fdata()

        for slice_idx in range(volume.shape[2]):
            image_slice = volume[:, :, slice_idx]
            mask_slice = segmentation[:, :, slice_idx]

            binary_mask = np.where((mask_slice == 1) | (mask_slice == 2), 1, 0).astype(np.uint8)

            if np.sum(binary_mask) == 0:
                continue

            img = image_slice - np.min(image_slice)
            img = (img / np.max(img) * 255).astype(np.uint8)
            img = cv2.resize(img, (224, 224))
            binary_mask = cv2.resize(binary_mask, (224, 224), interpolation=cv2.INTER_NEAREST)

            img_filename = f"{image_counter:05d}.png"
            cv2.imwrite(os.path.join(output_images_folder, img_filename), img)

            boxes = extract_bounding_boxes(binary_mask)

            label_filename = f"{image_counter:05d}.txt"
            with open(os.path.join(output_labels_folder, label_filename), 'w') as f:
                for box in boxes:
                    x_min, y_min, x_max, y_max = box
                    x_center = (x_min + x_max) / 2 / 224
                    y_center = (y_min + y_max) / 2 / 224
                    width = (x_max - x_min) / 224
                    height = (y_max - y_min) / 224
                    f.write(f"0 {x_center} {y_center} {width} {height}\n")

            image_counter += 1

print("Bounding box dataset created successfully ✅")

In [ ]:
!pip install torch torchvision --quiet

In [ ]:
import os
import torch
import numpy as np
from PIL import Image
import torchvision
from torchvision.transforms import functional as F

class TumorDataset(torch.utils.data.Dataset):
    def __init__(self, images_dir, labels_dir, transforms=None):
        self.images_dir = images_dir
        self.labels_dir = labels_dir
        self.transforms = transforms
        self.imgs = list(sorted(os.listdir(images_dir)))
        self.labels = list(sorted(os.listdir(labels_dir)))

    def __getitem__(self, idx):
        img_path = os.path.join(self.images_dir, self.imgs[idx])
        label_path = os.path.join(self.labels_dir, self.labels[idx])

        img = Image.open(img_path).convert("RGB")
        boxes = []
        labels = []

        with open(label_path) as f:
            for line in f:
                parts = line.strip().split()
                cls, x_center, y_center, width, height = map(float, parts)

                x_min = (x_center - width/2) * 224
                y_min = (y_center - height/2) * 224
                x_max = (x_center + width/2) * 224
                y_max = (y_center + height/2) * 224
                boxes.append([x_min, y_min, x_max, y_max])
                labels.append(int(cls))

        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        labels = torch.as_tensor(labels, dtype=torch.int64)

        target = {}
        target["boxes"] = boxes
        target["labels"] = labels

        if self.transforms:
            img = self.transforms(img)

        return img, target

    def __len__(self):
        return len(self.imgs)

In [ ]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

def get_model(num_classes):
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    return model

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model = get_model(num_classes=2)
model.to(device)

In [ ]:
import os

segmentation_dir = "/content/output/segmentations"
volume_dir = "/content/output/volume_pt1"

volume_files = sorted([f for f in os.listdir(volume_dir) if f.endswith('.nii')])

for volume_file in volume_files:
    volume_number = volume_file.split('-')[1].split('.')[0]
    segmentation_filename = f"segmentation-{volume_number}.nii"
    segmentation_path = os.path.join(segmentation_dir, segmentation_filename)

    if os.path.exists(segmentation_path):
        print(f"Found segmentation for {volume_file}: {segmentation_filename}")
    else:
        print(f"Segmentation not found for {volume_file}")

In [ ]:
import nibabel as nib
import numpy as np
import os
import cv2

volume_paths = [f"/content/output/volume_pt{i}" for i in range(1,6)]
segmentation_path = "/content/output/segmentations"

save_image_dir = "/content/rcnn_dataset/images"
save_label_dir = "/content/rcnn_dataset/labels"

os.makedirs(save_image_dir, exist_ok=True)
os.makedirs(save_label_dir, exist_ok=True)

def save_slice(image_slice, mask_slice, idx, base_filename):
    """Save 2D slice image and bounding box"""
    image_slice = (image_slice - np.min(image_slice)) / (np.max(image_slice) - np.min(image_slice) + 1e-8)
    image_slice = (image_slice * 255).astype(np.uint8)

    tumor_mask = np.logical_or(mask_slice == 1, mask_slice == 2).astype(np.uint8)

    contours, _ = cv2.findContours(tumor_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if len(contours) == 0:
        return

    image_filename = f"{base_filename}_{idx:03d}.png"
    label_filename = f"{base_filename}_{idx:03d}.txt"
    cv2.imwrite(os.path.join(save_image_dir, image_filename), image_slice)

    with open(os.path.join(save_label_dir, label_filename), "w") as f:
        for cnt in contours:
            x, y, w, h = cv2.boundingRect(cnt)
            if w > 5 and h > 5:
                f.write(f"{x} {y} {x+w} {y+h}\n")

for part_path in volume_paths:
    for file_name in os.listdir(part_path):
        if file_name.endswith('.nii') or file_name.endswith('.nii.gz'):
            volume_file = os.path.join(part_path, file_name)
            seg_file = os.path.join(segmentation_path, file_name)

            if not os.path.exists(seg_file):
                continue

            volume = nib.load(volume_file).get_fdata()
            seg = nib.load(seg_file).get_fdata()

            for idx in range(volume.shape[2]):
                image_slice = volume[:, :, idx]
                mask_slice = seg[:, :, idx]
                save_slice(image_slice, mask_slice, idx, base_filename=file_name.split('.')[0])

print("✅ Preprocessing finished. Images and labels saved!")

In [ ]:
import os
import nibabel as nib
import numpy as np
import cv2
from tqdm import tqdm

volume_dir = '/content/output/volume_pt1/'
segmentation_dir = '/content/output/segmentations/'
output_dir = '/content/output/sliced_images/'

os.makedirs(os.path.join(output_dir, 'Tumor'), exist_ok=True)
os.makedirs(os.path.join(output_dir, 'NoTumor'), exist_ok=True)

def save_slices(volume, segmentation, volume_filename):
    volume_number = volume_filename.split('-')[1].split('.')[0]

    num_slices = volume.shape[2]

    for i in range(num_slices):
        volume_slice = volume[:, :, i]
        segmentation_slice = segmentation[:, :, i]

        if np.any(segmentation_slice == 1):
            label = 'Tumor'
        else:
            label = 'NoTumor'

        slice_filename = f"{volume_number}_slice_{i}.png"
        slice_path = os.path.join(output_dir, label, slice_filename)

        volume_slice = cv2.normalize(volume_slice, None, 0, 255, cv2.NORM_MINMAX)
        volume_slice = volume_slice.astype(np.uint8)

        cv2.imwrite(slice_path, volume_slice)

for volume_filename in os.listdir(volume_dir):
    if volume_filename.endswith('.nii'):
        volume_path = os.path.join(volume_dir, volume_filename)
        volume_img = nib.load(volume_path)
        volume_data = volume_img.get_fdata()

        segmentation_filename = f"segmentation-{volume_filename.split('-')[1]}"
        segmentation_path = os.path.join(segmentation_dir, segmentation_filename)

        if os.path.exists(segmentation_path):
            segmentation_img = nib.load(segmentation_path)
            segmentation_data = segmentation_img.get_fdata()

            save_slices(volume_data, segmentation_data, volume_filename)
            print(f"Processed: {volume_filename}")
        else:
            print(f"Segmentation not found for {volume_filename}, skipping.")

In [ ]:
import os
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
from PIL import Image

volume_dir = '/content/output/volume_pt1'
segmentation_dir = '/content/output/segmentations'
output_dir = '/content/sliced_images'

os.makedirs(os.path.join(output_dir, 'Tumor'), exist_ok=True)
os.makedirs(os.path.join(output_dir, 'NoTumor'), exist_ok=True)

print(f"Volume directory: {volume_dir}")
print(f"Segmentation directory: {segmentation_dir}")
print(f"Output directory: {output_dir}")

for volume_filename in os.listdir(volume_dir):
    if volume_filename.endswith(".nii"):
        volume_path = os.path.join(volume_dir, volume_filename)

        volume_id = volume_filename.replace("volume-", "").replace(".nii", "")
        segmentation_path = os.path.join(segmentation_dir, f"segmentation-{volume_id}.nii")

        print(f"Processing volume: {volume_filename} with segmentation: {segmentation_path}")

        volume_nii = nib.load(volume_path)
        volume_data = volume_nii.get_fdata()
        segmentation_nii = nib.load(segmentation_path)
        segmentation_data = segmentation_nii.get_fdata()

        for i in range(volume_data.shape[2]):
            slice_data = volume_data[:, :, i]
            seg_slice_data = segmentation_data[:, :, i]

            if np.any(seg_slice_data > 0):
                label = 'Tumor'
            else:
                label = 'NoTumor'

            image_filename = f"{volume_filename.split('.')[0]}_slice_{i}.png"
            save_path = os.path.join(output_dir, label, image_filename)

            print(f"Saving {label} image: {save_path}")

            img = Image.fromarray(slice_data.astype(np.uint8))
            img.save(save_path)

print("Processing completed.")

In [ ]:
import os
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

volume_dir = '/content/output/volume_pt1'
segmentation_dir = '/content/output/segmentations'
output_dir = '/content/sliced_images'

tumor_dir = os.path.join(output_dir, "Tumor")
no_tumor_dir = os.path.join(output_dir, "NoTumor")
os.makedirs(tumor_dir, exist_ok=True)
os.makedirs(no_tumor_dir, exist_ok=True)

def normalize_image(img):
    img = img - np.min(img)
    img = img / (np.max(img) + 1e-8)
    img = (img * 255).astype(np.uint8)
    return img

for volume_filename in tqdm(os.listdir(volume_dir), desc="Processing volumes"):
    if volume_filename.endswith(".nii"):
        volume_path = os.path.join(volume_dir, volume_filename)

        volume_id = volume_filename.replace("volume-", "").replace(".nii", "")
        segmentation_path = os.path.join(segmentation_dir, f"segmentation-{volume_id}.nii")

        if not os.path.exists(segmentation_path):
            print(f"Segmentation not found for {volume_filename}, skipping.")
            continue

        volume_nii = nib.load(volume_path)
        volume_data = volume_nii.get_fdata()

        segmentation_nii = nib.load(segmentation_path)
        segmentation_data = segmentation_nii.get_fdata()

        num_slices = volume_data.shape[2]
        for i in range(num_slices):
            slice_img = volume_data[:, :, i]
            slice_seg = segmentation_data[:, :, i]

            if np.max(slice_img) == 0:
                continue

            slice_img = normalize_image(slice_img)

            if np.any(slice_seg == 1) or np.any(slice_seg == 2):
                save_path = os.path.join(tumor_dir, f"{volume_id}_slice_{i}.png")
            else:
                save_path = os.path.join(no_tumor_dir, f"{volume_id}_slice_{i}.png")

            plt.imsave(save_path, slice_img, cmap='gray')

print("✅ Done saving slices!")

In [ ]:
import random
import matplotlib.pyplot as plt
import os
import matplotlib.image as mpimg

tumor_dir = '/content/sliced_images/Tumor'
no_tumor_dir = '/content/sliced_images/NoTumor'

tumor_samples = random.sample(os.listdir(tumor_dir), min(5, len(os.listdir(tumor_dir))))
no_tumor_samples = random.sample(os.listdir(no_tumor_dir), min(5, len(os.listdir(no_tumor_dir))))

fig, axes = plt.subplots(2, 5, figsize=(20, 8))

for idx, img_name in enumerate(tumor_samples):
    img = mpimg.imread(os.path.join(tumor_dir, img_name))
    axes[0, idx].imshow(img, cmap='gray')
    axes[0, idx].set_title(f"Tumor: {img_name}")
    axes[0, idx].axis('off')

for idx, img_name in enumerate(no_tumor_samples):
    img = mpimg.imread(os.path.join(no_tumor_dir, img_name))
    axes[1, idx].imshow(img, cmap='gray')
    axes[1, idx].set_title(f"No Tumor: {img_name}")
    axes[1, idx].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
import os
import numpy as np
import torch
import torchvision
from torch.utils.data import Dataset
from torchvision.io import read_image

class TumorDataset(Dataset):
    def __init__(self, image_dir, transforms=None):
        self.image_dir = image_dir
        self.transforms = transforms
        self.images = list(sorted(os.listdir(image_dir)))

    def __getitem__(self, idx):
        img_path = os.path.join(self.image_dir, self.images[idx])
        img = read_image(img_path).float() / 255.0

        label = 1 if "Tumor" in self.images[idx] else 0

        target = {}
        target['labels'] = torch.tensor([label], dtype=torch.int64)
        target['boxes'] = torch.tensor([[0, 0, img.shape[2], img.shape[1]]], dtype=torch.float32)
        target['image_id'] = torch.tensor([idx])

        if self.transforms:
            img = self.transforms(img)

        return img, target

    def __len__(self):
        return len(self.images)

In [ ]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

in_features = model.roi_heads.box_predictor.cls_score.in_features

model.roi_heads.box_predictor = FastRCNNPredictor(in_features, 2)

In [ ]:
train_loader = DataLoader(dataset, batch_size=2, shuffle=True, collate_fn=lambda x: tuple(zip(*x)))